# Week 1 - Battle of the Neighborhoods

## 1-Background and Problem discussion
**Clearly define a problem or an idea of your choice, where you would need to leverage the Foursquare location data to solve or execute. Remember that data science problems always target an audience and are meant to help a group of stakeholders solve a problem, so make sure that you explicitly describe your audience and why they would care about your problem.**

The goal of this project is to help a couple moving to Toronto find the gym/fitness center that has what they're looking for (high-end equipment, sauna, pool, steam room, etc).  They are a high-income couple and have discovered that in the past, the high-income neighborhoods offer the gym/fitness centers with these amenities, which means that the membership price is also high.

## 2-Data description and usefulness
**Describe the data that you will be using to solve the problem or execute your idea. Remember that you will need to use the Foursquare location data to solve the problem or execute your idea. You can absolutely use other datasets in combination with the Foursquare location data. So make sure that you provide adequate explanation and discussion, with examples, of the data that you will be using, even if it is only Foursquare location data.**

In order to help this couple make the best decision, I will be using the Toronto 2016 Census data to determine which neighborhoods have the higher incomes.  Then by using the Foursquare API, I will be able to locate the gym/fitness centers within these neighborhoods and hopefully determine the amenities that they offer.

In [4]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from IPython.display import Image
from IPython.core.display import HTML

import requests
from pandas.io.json import json_normalize

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

!conda install -c conda-forge folium=0.5.0 --yes
import folium

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

print("Imported LIbraries")

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    openssl-1.1.1e             |       h516909a_0         2.1 MB  conda-forge
    certifi-2019.11.28         |   py36h9f0ad1d_1         149 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1

In [5]:
csv_path='https://www.toronto.ca/ext/open_data/catalog/data_set_files/2016_neighbourhood_profiles.csv'
df = pd.read_csv(csv_path,encoding='latin1')
print('Data loaded')

Data loaded


#### Data Sample

In [6]:
df.head()

,Category,Topic,Data Source,Characteristic,City of Toronto,Agincourt North,Agincourt South-Malvern West,Alderwood,Annex,Banbury-Don Mills,Bathurst Manor,Bay Street Corridor,Bayview Village,Bayview Woods-Steeles,Bedford Park-Nortown,Beechborough-Greenbrook,Bendale,Birchcliffe-Cliffside,Black Creek,Blake-Jones,Briar Hill-Belgravia,Bridle Path-Sunnybrook-York Mills,Broadview North,Brookhaven-Amesbury,Cabbagetown-South St. James Town,Caledonia-Fairbank,Casa Loma,Centennial Scarborough,Church-Yonge Corridor,Clairlea-Birchmount,Clanton Park,Cliffcrest,Corso Italia-Davenport,Danforth,Danforth East York,Don Valley Village,Dorset Park,Dovercourt-Wallace Emerson-Junction,Downsview-Roding-CFB,Dufferin Grove,East End-Danforth,Edenbridge-Humber Valley,Eglinton East,Elms-Old Rexdale,Englemount-Lawrence,Eringate-Centennial-West Deane,Etobicoke West Mall,Flemingdon Park,Forest Hill North,Forest Hill South,Glenfield-Jane Heights,Greenwood-Coxwell,Guildwood,Henry Farm,High Park North,High Park-Swansea,Highland Creek,Hillcrest Village,Humber Heights-Westmount,Humber Summit,Humbermede,Humewood-Cedarvale,Ionview,Islington-City Centre West,Junction Area,Keelesdale-Eglinton West,Kennedy Park,Kensington-Chinatown,Kingsview Village-The Westway,Kingsway South,Lambton Baby Point,L'Amoreaux,Lansing-Westgate,Lawrence Park North,Lawrence Park South,Leaside-Bennington,Little Portugal,Long Branch,Malvern,Maple Leaf,Markland Wood,Milliken,Mimico (includes Humber Bay Shores),Morningside,Moss Park,Mount Dennis,Mount Olive-Silverstone-Jamestown,Mount Pleasant East,Mount Pleasant West,New Toronto,Newtonbrook East,Newtonbrook West,Niagara,North Riverdale,North St. James Town,Oakridge,Oakwood Village,O'Connor-Parkview,Old East York,Palmerston-Little Italy,Parkwoods-Donalda,Pelmo Park-Humberlea,Playter Estates-Danforth,Pleasant View,Princess-Rosethorn,Regent Park,Rexdale-Kipling,Rockcliffe-Smythe,Roncesvalles,Rosedale-Moore Park,Rouge,Runnymede-Bloor West Village,Rustic,Scarborough Village,South Parkdale,South Riverdale,St.Andrew-Windfields,Steeles,Stonegate-Queensway,Tam O'Shanter-Sullivan,Taylor-Massey,The Beaches,Thistletown-Beaumond Heights,Thorncliffe Park,Trinity-Bellwoods,University,Victoria Village,Waterfront Communities-The Island,West Hill,West Humber-Clairville,Westminster-Branson,Weston,Weston-Pelham Park,Wexford/Maryvale,Willowdale East,Willowdale West,Willowridge-Martingrove-Richview,Woburn,Woodbine Corridor,Woodbine-Lumsden,Wychwood,Yonge-Eglinton,Yonge-St.Clair,York University Heights,Yorkdale-Glen Park
0,Neighbourhood Information,Neighbourhood Information,City of Toronto,Neighbourhood Number,NaN,129,128,20,95,42,34,76,52,49,39,112,127,122,24,69,108,41,57,30,71,109,96,133,75,120,33,123,92,66,59,47,126,93,26,83,62,9,138,5,32,11,13,44,102,101,25,65,140,53,88,87,134,48,8,21,22,106,125,14,90,110,124,78,6,15,114,117,38,105,103,56,84,19,132,29,12,130,17,135,73,115,2,99,104,18,50,36,82,68,74,121,107,54,58,80,45,23,67,46,10,72,4,111,86,98,131,89,28,139,85,70,40,116,16,118,61,63,3,55,81,79,43,77,136,1,35,113,91,119,51,37,7,137,64,60,94,100,97,27,31
1,Neighbourhood Information,Neighbourhood Information,City of Toronto,TSNS2020 Designation,NaN,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,NIA,No Designation,No Designation,NIA,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,Emerging Neighbourhood,No Designation,NIA,No Designation,No Designation,No Designation,NIA,NIA,Emerging Neighbourhood,No Designation,No Designation,NIA,No Designation,No Designation,NIA,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,Emerging Neighbourhood,NIA,NIA,No Designation,NIA,No Designation,No Designation,NIA,NIA,No Designat

#### Gathering Neighbourhood names

In [7]:
Neighbourhoods = list(df.columns.values)
Neighbourhoods = Neighbourhoods[5:]
print(Neighbourhoods)

['Agincourt North', 'Agincourt South-Malvern West', 'Alderwood', 'Annex', 'Banbury-Don Mills', 'Bathurst Manor', 'Bay Street Corridor', 'Bayview Village', 'Bayview Woods-Steeles', 'Bedford Park-Nortown', 'Beechborough-Greenbrook', 'Bendale', 'Birchcliffe-Cliffside', 'Black Creek', 'Blake-Jones', 'Briar Hill-Belgravia', 'Bridle Path-Sunnybrook-York Mills', 'Broadview North', 'Brookhaven-Amesbury', 'Cabbagetown-South St. James Town', 'Caledonia-Fairbank', 'Casa Loma', 'Centennial Scarborough', 'Church-Yonge Corridor', 'Clairlea-Birchmount', 'Clanton Park', 'Cliffcrest', 'Corso Italia-Davenport', 'Danforth', 'Danforth East York', 'Don Valley Village', 'Dorset Park', 'Dovercourt-Wallace Emerson-Junction', 'Downsview-Roding-CFB', 'Dufferin Grove', 'East End-Danforth', 'Edenbridge-Humber Valley', 'Eglinton East', 'Elms-Old Rexdale', 'Englemount-Lawrence', 'Eringate-Centennial-West Deane', 'Etobicoke West Mall', 'Flemingdon Park', 'Forest Hill North', 'Forest Hill South', 'Glenfield-Jane Heig

### Collecting Neighourhood Population

In [8]:
Toronto_df = pd.DataFrame(index=Neighbourhoods, columns=["Income_2016"])
Toronto_df.head()

,Income_2016
Agincourt North,NaN
Agincourt South-Malvern West,NaN
Alderwood,NaN
Annex,NaN
Banbury-Don Mills,NaN


In [9]:
for index, row in Toronto_df.iterrows():
    Toronto_df.at[index, 'Population_2016'] = df[index][2]
    Toronto_df.at[index, 'Income_2016'] = df[index][2264]
    
Toronto_df.sort_values('Income_2016')

,Income_2016,Population_2016
St.Andrew-Windfields,"100,516","17,812"
Edenbridge-Humber Valley,"101,551","15,535"
Lawrence Park North,"111,730","14,607"
Annex,"112,766","30,526"
Yonge-St.Clair,"114,174","12,528"
Bedford Park-Nortown,"123,077","23,236"
Leaside-Bennington,"125,564","16,828"
Kingsway South,"144,642","9,271"
Casa Loma,"165,047","10,968"
Lawrence Park South,"169,203","15,179"


In [10]:
CLIENT_ID = 'RM5123RDIEIDRDWEXMUEVY2YQFI5HYX1CFVHFBKUDG3CO3BI' # your Foursquare ID
CLIENT_SECRET = 'LFO3QP32TKIVRAQ1YDQSNDUUTVSZ5JEAV4NMNRX35UJOZEHZ' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('RM5123RDIEIDRDWEXMUEVY2YQFI5HYX1CFVHFBKUDG3CO3BI: ' + CLIENT_ID)
print('LFO3QP32TKIVRAQ1YDQSNDUUTVSZ5JEAV4NMNRX35UJOZEHZ:' + CLIENT_SECRET)

Your credentails:
RM5123RDIEIDRDWEXMUEVY2YQFI5HYX1CFVHFBKUDG3CO3BI: RM5123RDIEIDRDWEXMUEVY2YQFI5HYX1CFVHFBKUDG3CO3BI
LFO3QP32TKIVRAQ1YDQSNDUUTVSZ5JEAV4NMNRX35UJOZEHZ:LFO3QP32TKIVRAQ1YDQSNDUUTVSZ5JEAV4NMNRX35UJOZEHZ


In [11]:
address = 'Toronto, ON, Canada'

geolocator = Nominatim(user_agent="Foursquare_Agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

43.653963 -79.387207


In [28]:
search_query = 'Gym'
radius = 500
print(search_query + '...OK!')

Gym...OK!


In [29]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=RM5123RDIEIDRDWEXMUEVY2YQFI5HYX1CFVHFBKUDG3CO3BI&client_secret=LFO3QP32TKIVRAQ1YDQSNDUUTVSZ5JEAV4NMNRX35UJOZEHZ&ll=43.653963,-79.387207&v=20180604&query=Gym&radius=500&limit=30'

In [30]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e7a305378a484001b8cca36'},
 'response': {'venues': [{'id': '5133f680e4b02e871367c60c',
    'name': 'The Gym at the Shangri-La',
    'location': {'lat': 43.648773826131794,
     'lng': -79.38651748319191,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.648773826131794,
       'lng': -79.38651748319191}],
     'distance': 580,
     'cc': 'CA',
     'country': 'Canada',
     'formattedAddress': ['Canada']},
    'categories': [{'id': '4bf58dd8d48988d176941735',
      'name': 'Gym',
      'pluralName': 'Gyms',
      'shortName': 'Gym',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/gym_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1585066137',
    'hasPerk': False},
   {'id': '4cd044c29d87224bf129543b',
    'name': 'University Centre Gym',
    'location': {'lat': 43.653571,
     'lng': -79.386979,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.653571,
       'lng': -79.

In [31]:
venues = results['response']['venues']

df = json_normalize(venues)
df.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",False,5133f680e4b02e871367c60c,NaN,CA,NaN,Canada,NaN,580,[Canada],"[{'label': 'display', 'lat': 43.64877382613179...",43.648774,-79.386517,NaN,NaN,The Gym at the Shangri-La,v-1585066137
1,"[{'id': '4bf58dd8d48988d175941735', 'name': 'G...",False,4cd044c29d87224bf129543b,NaN,CA,NaN,Canada,NaN,47,[Canada],"[{'label': 'display', 'lat': 43.653571, 'lng':...",43.653571,-79.386979,NaN,NaN,University Centre Gym,v-1585066137
2,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",False,4f61e092e4b0d7325fb1f8a3,145 Richmond Street W,CA,Toronto,Canada,at University Ave.,457,"[145 Richmond Street W (at University Ave.), T...","[{'label': 'display', 'lat': 43.64999445230568...",43.649994,-79.385730,NaN,ON,Hilton Gym,v-1585066137
3,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",False,5325c1c9498e8e967d1b7816,140 Simcoe Street,CA,Toronto,Canada,Richmond,485,"[140 Simcoe Street (Richmond), Toronto ON, Can...","[{'label': 'display', 'lat': 43.64961242675781...",43.649612,-79.386810,NaN,ON,Gym,v-1585066137
4,"[{'id': '4d954b06a243a5684965b473', 'name': 'R...",False,54229196498e92c87216e7e8,NaN,CA,NaN,Canada,NaN,299,[Canada],"[{'label': 'display', 'lat': 43.65545242320724...",43.655452,-79.384114,NaN,NaN,The Gym @ Motion,v-1585066137


In [ ]:
gym_map = folium.Map(location=[latitude, longitude], zoom_start=13)

folium.features.CircleMarker(
    [latitude, longitude],
    radius = 10,
    color='red',
    popup='Toronto',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(gym_map)

for lat, lng, label in zip (df.lat, df.lng):
    folium.feautres.CircleMarker(
        [lat, lng],
        radius=5,
        color='green',
        popup=label,
        fill = True,
        fill_color='green',
        fill_opacity=0.6).add_to(gym_map)

gym_map

# Results Section

#### During this project, what results I noted using the Data is that there are only 4 locations that populate upon using the Foursquare API.  This makes it relatively simple for the couple to make a selection.

# Discussion Section

#### I noted that only 4 selections appeared using the API.  It's possible, since Foursquare was where the data was pulled from, that no other fitness centers in Toronto were mentioned. My recommendations would be for the couple to dive in to the 4 options that populated and see which one meets their needs.  I would recommend looking into "The Gym at the Shangri La."

# Conclusion Section

#### In conclusion, while using machine learning to scrape the data to determine the income and population of Toronto was very useful, I feel that there are better ways to determine which fitness centers meet the couple's requirements.